📄 .gitattributes
📄 .gitignore
📄 LICENSE
📄 README (3).md
📄 README.md
📁 backend
    📁 api
        📄 upload.py
    📄 config.py
    📄 main.py
    📁 models
        📄 skin_disease_model.h5
    📁 services
        📄 image_classifier.py
        📄 symptom_to_disease.py
    📁 utils
        📄 filtering_with_ner.py
        📄 image_preprocessing.py
        📄 text_cleaning.py
📁 data
    📁 Vector
        📁 symptom_faiss_db
            📄 index.faiss
            📄 index.pkl
    📄 labels.json
    📄 mayo_diseases.csv
    📄 test_symptom_cases.csv
📁 evaluation
    📄 evaluate_rag_symptom.py
📁 frontend
    📄 index.html
📄 logo.png
📁 notebook
    📄 llm.ipynb
    📄 skin_disease_prediction.ipynb
    📄 web_scrapping.ipynb
📄 requirements.txt
📁 scripts
    📄 model.py
    📁 scrapers
        📄 main.py
        📄 web_scraper2.py
        📄 web_scrapers.py
    📄 symptoms_to_vectordb.py
📄 transfaer.ipynb


In [1]:
import re
import json
import os
import sys